In [63]:
import pandas as pd

In [64]:
df = pd.read_csv("../nba_games.csv", index_col=0)

In [65]:
df = df.sort_values("date")

In [66]:
df = df.reset_index(drop=True)

In [67]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

Want to predict how the team did in the next game. So pull the won column from the next game and move it back one row.

In [68]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/6v/tkqp5lhn4l90pnrnv57z_cnw0000gn/T/ipykernel_12192/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/6v/tkqp5lhn4l90pnrnv57z_cnw0000gn/T/ipykernel_12192/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/6v/tkqp5lhn4l90pnrnv57z_cnw0000gn/T/ipykernel_12192/1127067056.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [69]:
df[df["team"] == "WAS"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
11,240.0,33.0,84.0,0.393,7.0,28.0,0.250,15.0,25.0,0.600,...,25.2,134.0,98.0,ORL,87,1,2016,2015-10-28,True,True
63,240.0,35.0,68.0,0.515,12.0,21.0,0.571,36.0,44.0,0.818,...,32.4,138.0,122.0,MIL,113,1,2016,2015-10-30,True,False
69,240.0,38.0,90.0,0.422,6.0,22.0,0.273,28.0,33.0,0.848,...,28.7,153.0,108.0,NYK,117,0,2016,2015-10-31,False,True
124,240.0,42.0,87.0,0.483,8.0,23.0,0.348,10.0,14.0,0.714,...,30.0,160.0,109.0,SAS,99,0,2016,2015-11-04,True,False
161,240.0,36.0,88.0,0.409,8.0,25.0,0.320,18.0,23.0,0.783,...,41.6,146.0,103.0,BOS,118,1,2016,2015-11-06,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17471,240.0,38.0,82.0,0.463,8.0,21.0,0.381,18.0,20.0,0.900,...,34.4,245.0,116.0,BOS,144,1,2022,2022-04-03,False,True
17483,240.0,56.0,101.0,0.554,13.0,30.0,0.433,7.0,14.0,0.500,...,32.8,168.0,132.0,MIN,114,1,2022,2022-04-05,True,False
17510,240.0,42.0,86.0,0.488,10.0,35.0,0.286,9.0,11.0,0.818,...,60.2,159.0,114.0,ATL,118,1,2022,2022-04-06,False,False
17538,240.0,35.0,69.0,0.507,9.0,28.0,0.321,13.0,17.0,0.765,...,35.5,135.0,104.0,NYK,114,0,2022,2022-04-08,False,False


For all the null values in the target column, meaning there is no next game, set those to 2

In [70]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/6v/tkqp5lhn4l90pnrnv57z_cnw0000gn/T/ipykernel_12192/1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/6v/tkqp5lhn4l90pnrnv57z_cnw0000gn/T/ipykernel_12192/161839

Turn True to 1 False to 0 and Null to 2

In [71]:
df["target"] = df["target"].astype(int, errors="ignore")

In [72]:
df["target"].value_counts()

target
1    8872
0    8870
2      30
Name: count, dtype: int64

In [73]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]

Delete columns with nulls

In [74]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [75]:
df = df[valid_columns].copy()

For most machine learning algorithms need a smaller number of columns. Use a feature selector to do this. Time series split useful to use old data to predict newer data.

In [76]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

Scale the columns. Ridge regression works best when columns are scaled. We are going to scale each column between 0 and 1. Cant scale certain columns.

In [77]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [78]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [79]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [80]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

If the feature select has picked the feature .get_support will be true. So we want to find the columns the selector picked

In [83]:
predictors = list(selected_columns[sfs.get_support()])

Split up data by season and use past season to predict future seasons. Need 2 seasons of data to make predictons about future season

In [86]:
def backtest(data, model, predictors, start=2, step=1):
    # list of dataframes where each df is predictions for a season
    all_predictions = []

    seasons = sorted(data["season"].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        # Taining data is all data from previous seasons
        train = data[data["season"] < season]

        # Testing data is all data from the current season
        test = data[data["season"] == season]

        # Fit the model to the training data
        model.fit(train[predictors], train["target"])

        # Make predictions on the testing data
        preds = model.predict(test[predictors])

        # Make the predictions a pandas series with the same index as the testing data
        preds = pd.Series(preds, index=test.index)

        # combine the actual and predicted values
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        # Add the combined data to the list of all predictions
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [87]:
predictions = backtest(df, rr, predictors)

In [90]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["prediction"])

0.5485110470701249